Data Analysis for cycle wands project
Code will use OS NGD API to obtain data on cycle lanes in the UK
Analysis will be conducted to split cycle lanes by type and location (London Boroughs)

In [ ]:
!pip install -r requirements.txt
import requests
import geopandas as gpd
import pandas as pd


In [ ]:
import requests
import json

# API features
api_key = "iKiHPsM65ooKNn5q48AgTlgJbkpmen6h"
collection_id = "trn-ntwk-cyclelane-1"
os_ngd_url = f"https://api.os.uk/features/ngd/ofa/v1/collections/{collection_id}/items"s
# Parameters
request_parameters = {
    "key": api_key,
    "bbox": "-0.5202136724,51.3257416338,0.2584423334,51.6941179174",
    "limit": 100
}
os_response = requests.get(url = os_ngd_url, headers={'key': api_key}, params=request_parameters, timeout=40).json()

print("Number returned:", os_response.get("numberReturned"))
print("Number matched:", os_response.get("numberMatched"))

request_parameters = {
    "limit": 10000
}

os_response = requests.get(
    url=f"https://api.os.uk/features/ngd/ofa/v1/collections/{collection_id}/items",
    headers={'key': api_key},
    params=request_parameters,
    timeout=20
).json()

features = os_response.get('features', [])
print("Total features returned:", len(features))


Number returned: 5
Number matched: None
Total features returned: 100
